In [4]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cooking = pd.read_csv("/content/drive/MyDrive/Projectdata/Cooking_data_set.csv")
error = pd.read_csv("/content/drive/MyDrive/Projectdata/Error_Message.csv")
product = pd.read_csv("/content/drive/MyDrive/Projectdata/Product_Name.csv")
booking = pd.read_csv("/content/drive/MyDrive/Projectdata/booking_data.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
cooking_error= cooking.merge(error, left_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명'], right_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명'], how='outer')
cooking_error

## cooking과 error를 merge를 한다

순번 생산라인코드      생산라인명        생산일자      품목코드              품목명 작업장코드  \
0       3   C001      쿠커 #1  2011-01-03  23K01312       서비스면양념장(재)  W003   
1       1   C002      쿠커 #2  2011-01-03  23V00019   고구마샐러드믹스베이스(재)  W003   
2      13   C003      쿠커 #3  2011-01-03  1.#JE+00    이마트가쓰오다시국물(재)  W003   
3       7   C005   2동 쿠커 #4  2011-01-03  26A01006   고구마샐러드바145g(재)  W003   
4       2   C007     프리머 #1  2011-01-03  26B06024  이츠웰골드마요네즈1kg(재)  W003   
...    ..    ...        ...         ...       ...              ...   ...   
24066   1   D088  냉동밥 내포장#1  2021-10-29  1BA03713              NaN  W005   
24067   2   D088  냉동밥 내포장#1  2021-10-29  1BA03713              NaN  W005   
24068   3   D088  냉동밥 내포장#1  2021-10-29  1BA03713              NaN  W005   
24069   4   D088  냉동밥 내포장#1  2021-10-29  1BA03606         콩콩푸드밥3kg  W005   
24070   5   P003  파우치포장(#3)  2021-10-29  16B06118    주니어쿱마요네즈(1kg)  W007   

           충전실온도        실링온도        쿠킹온도  쿠킹스팀압력     실링압력   생산시간 발생시간 종료시간  \
0      71.669853   96.769853  131.742003  23.315  201.844  70.37  NaN  NaN   
1      70.386480  102.086480  136.926080  23.226  212.043  70.83  NaN  NaN   
2      71.206130  112.306130  147.579680  23.516  208.781  73.55  NaN  NaN   
3      70.430565   96.030565  131.569165  23.692  208.269  74.81  NaN  NaN   
4      72.179319  114.079318  149.928068  23.899  211.625  73.40  NaN  NaN   
...          ...         ...         ...     ...      ...    ...  ...  ...   
24066  72.136538  117.336538  152.813288  23.651  203.823  71.54  NaN  NaN   
24067  71.874219  106.174219  141.470619  23.531  207.574  76.03  NaN  NaN   
24068  72.942705  119.742705  155.733955  23.994  210.006  74.50  NaN  NaN   
24069  70.664452   65.764452  100.773102  23.339  209.513  72.12  NaN  NaN   
24070  71.015225  106.115225  141.148725  23.356  205.288  76.03  NaN  NaN   

       오류조치시간 비가동코드 Error_Message 오류발생작업장명  
0         NaN   NaN           NaN      NaN  
1         NaN   NaN           NaN      NaN  
2         NaN   NaN           NaN      NaN  
3         NaN   NaN           NaN      NaN  
4         NaN   NaN           NaN      NaN  
...       ...   ...           ...      ...  
24066     NaN   NaN           NaN      NaN  
24067     NaN   NaN           NaN      NaN  
24068     NaN   NaN           NaN      NaN  
24069     NaN   NaN           NaN      NaN  
24070     NaN   NaN           NaN      NaN  

[24071 rows x 19 columns]

In [7]:
cooking_error_set = cooking_error.drop(['순번','생산라인코드','품목코드','생산라인명','생산일자','품목코드', '품목명','작업장코드','발생시간','종료시간','오류조치시간','비가동코드','오류발생작업장명','생산시간'], axis=1)
cooking_error_set

# 쓸모없는 컬럼 제거 후 cooking_error_set에 할당

충전실온도        실링온도        쿠킹온도  쿠킹스팀압력     실링압력 Error_Message
0      71.669853   96.769853  131.742003  23.315  201.844           NaN
1      70.386480  102.086480  136.926080  23.226  212.043           NaN
2      71.206130  112.306130  147.579680  23.516  208.781           NaN
3      70.430565   96.030565  131.569165  23.692  208.269           NaN
4      72.179319  114.079318  149.928068  23.899  211.625           NaN
...          ...         ...         ...     ...      ...           ...
24066  72.136538  117.336538  152.813288  23.651  203.823           NaN
24067  71.874219  106.174219  141.470619  23.531  207.574           NaN
24068  72.942705  119.742705  155.733955  23.994  210.006           NaN
24069  70.664452   65.764452  100.773102  23.339  209.513           NaN
24070  71.015225  106.115225  141.148725  23.356  205.288           NaN

[24071 rows x 6 columns]

In [8]:
cooking_error_set.isnull().sum()

충전실온도                0
실링온도                28
쿠킹온도                28
쿠킹스팀압력               0
실링압력                 0
Error_Message    22991
dtype: int64

In [9]:
cooking_error_set['error'] = cooking_error_set['Error_Message'].isnull()
cooking_error_set

# error 컬럼을 만들어줌 
# error컬럼은 Error_Message의 값이 존재하면 1, 즉 불량품
# error컬럼은 Error_Message의 값이 없으면 0, 상품

충전실온도        실링온도        쿠킹온도  쿠킹스팀압력     실링압력 Error_Message  error
0      71.669853   96.769853  131.742003  23.315  201.844           NaN   True
1      70.386480  102.086480  136.926080  23.226  212.043           NaN   True
2      71.206130  112.306130  147.579680  23.516  208.781           NaN   True
3      70.430565   96.030565  131.569165  23.692  208.269           NaN   True
4      72.179319  114.079318  149.928068  23.899  211.625           NaN   True
...          ...         ...         ...     ...      ...           ...    ...
24066  72.136538  117.336538  152.813288  23.651  203.823           NaN   True
24067  71.874219  106.174219  141.470619  23.531  207.574           NaN   True
24068  72.942705  119.742705  155.733955  23.994  210.006           NaN   True
24069  70.664452   65.764452  100.773102  23.339  209.513           NaN   True
24070  71.015225  106.115225  141.148725  23.356  205.288           NaN   True

[24071 rows x 7 columns]

In [10]:
cooking_error_set = cooking_error_set.drop('Error_Message', axis=1)
cooking_error_set
# 이제 필요없어진 Error_Message 컬럼을 제거

충전실온도        실링온도        쿠킹온도  쿠킹스팀압력     실링압력  error
0      71.669853   96.769853  131.742003  23.315  201.844   True
1      70.386480  102.086480  136.926080  23.226  212.043   True
2      71.206130  112.306130  147.579680  23.516  208.781   True
3      70.430565   96.030565  131.569165  23.692  208.269   True
4      72.179319  114.079318  149.928068  23.899  211.625   True
...          ...         ...         ...     ...      ...    ...
24066  72.136538  117.336538  152.813288  23.651  203.823   True
24067  71.874219  106.174219  141.470619  23.531  207.574   True
24068  72.942705  119.742705  155.733955  23.994  210.006   True
24069  70.664452   65.764452  100.773102  23.339  209.513   True
24070  71.015225  106.115225  141.148725  23.356  205.288   True

[24071 rows x 6 columns]

In [11]:
cooking_error_set.isnull().sum()
# 실링온도, 쿠킹온도값이 없으면 의미 없는 row이기 때문에 삭제를 해준다

충전실온도      0
실링온도      28
쿠킹온도      28
쿠킹스팀압력     0
실링압력       0
error      0
dtype: int64

In [12]:
cooking_error_set = cooking_error_set.dropna()
# 결측치 제거 

In [13]:
cooking_error_set.dtypes

충전실온도     float64
실링온도      float64
쿠킹온도      float64
쿠킹스팀압력    float64
실링압력      float64
error        bool
dtype: object

In [14]:
cooking_error_set['충전실온도'] = cooking_error_set['충전실온도'].astype({'충전실온도' : 'int'})
cooking_error_set['실링온도'] = cooking_error_set['실링온도'].astype({'실링온도' : 'int'})
cooking_error_set['쿠킹온도'] = cooking_error_set['쿠킹온도'].astype({'쿠킹온도' : 'int'})
cooking_error_set['쿠킹스팀압력'] = cooking_error_set['쿠킹스팀압력'].astype({'쿠킹스팀압력' : 'int'})
cooking_error_set['실링압력'] = cooking_error_set['실링압력'].astype({'실링압력' : 'int'})
cooking_error_set['error'] = cooking_error_set['error'].astype({'error' : 'int'})
cooking_error_set.dtypes

# float64 에서 int로 값 변환 필요 Classifier에선 정수만 사용가능 
# error컬럼은 bool에서 int로 변환

<ipython-input-14-f3a825e99544>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cooking_error_set['충전실온도'] = cooking_error_set['충전실온도'].astype({'충전실온도' : 'int'})
<ipython-input-14-f3a825e99544>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cooking_error_set['실링온도'] = cooking_error_set['실링온도'].astype({'실링온도' : 'int'})
<ipython-input-14-f3a825e99544>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

충전실온도     int64
실링온도      int64
쿠킹온도      int64
쿠킹스팀압력    int64
실링압력      int64
error     int64
dtype: object

In [15]:
cooking_error_set

충전실온도  실링온도  쿠킹온도  쿠킹스팀압력  실링압력  error
0         71    96   131      23   201      1
1         70   102   136      23   212      1
2         71   112   147      23   208      1
3         70    96   131      23   208      1
4         72   114   149      23   211      1
...      ...   ...   ...     ...   ...    ...
24066     72   117   152      23   203      1
24067     71   106   141      23   207      1
24068     72   119   155      23   210      1
24069     70    65   100      23   209      1
24070     71   106   141      23   205      1

[24043 rows x 6 columns]

In [16]:
x = cooking_error_set.drop('error',axis=1)
y = cooking_error_set['error']

# x축에 error를 제외한 데이터 값을 넣어준다
# y축에 error컬럼만 넣어준다.

In [23]:
GradientBoostingClassifier,
x_tn, x_te, y_tn, y_te = train_test_split(x,y, test_size=0.3, random_state=0) # 학습시킬 데이터, 테스트용 데이터 설정


# 스케일링 하는 곳
std_scale = StandardScaler()
std_scale.fit(x_tn)
x_tn_std = std_scale.transform(x_tn)
x_te_std = std_scale.transform(x_te)

# 모델생성 randomforest로 만들어줌
# clf_rf = RandomForestClassifier(max_depth=2, random_state=0)
# clf_rf.fit(x_tn_std,y_tn)

# 모델생성 Grandient Boosting 버전
clf_rf = GradientBoostingClassifier(random_state=0, learning_rate=0.02)
clf_rf.fit(x_tn, y_tn)

GradientBoostingClassifier(learning_rate=0.02, random_state=0)

In [24]:
# 예측값 class에 들어가는 예측값을 넣는다.
pred_rf = clf_rf.predict(x_te_std)
print(pred_rf)

[0 0 0 ... 0 0 0]


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [25]:
# 정확도 검사
accuracy = accuracy_score(y_te, pred_rf)
print(accuracy)

0.04491889643698877
